In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lung_Cancer"
cohort = "GSE248830"

# Input paths
in_trait_dir = "../../input/GEO/Lung_Cancer"
in_cohort_dir = "../../input/GEO/Lung_Cancer/GSE248830"

# Output paths
out_data_file = "../../output/preprocess/Lung_Cancer/GSE248830.csv"
out_gene_data_file = "../../output/preprocess/Lung_Cancer/gene_data/GSE248830.csv"
out_clinical_data_file = "../../output/preprocess/Lung_Cancer/clinical_data/GSE248830.csv"
json_path = "../../output/preprocess/Lung_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Unlocking Molecular mechanisms and identifying druggable targets in matched-paired brain metastasis of Breast and Lung cancers "
!Series_summary	"Introduction: The incidence of brain metastases in cancer patients is increasing, with lung and breast cancer being the most common sources. Despite advancements in targeted therapies, the prognosis remains poor, highlighting the importance to investigate the underlying mechanisms in brain metastases. The aim of this study was to investigate the differences in the molecular mechanisms involved in brain metastasis of breast and lung cancers. In addition, we aimed to identify cancer lineage-specific druggable targets in the brain metastasis. Methods: To that aim, a cohort of 44 FFPE tissue samples, including 22 breast cancer and 22 lung adenocarcinoma (LUAD) and their matched-paired brain metastases were collected. Targeted gene expression profiles of primary tumors were compared to their matched-paired br

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Step 1: Gene Expression Data Availability
is_gene_available = True # The dataset appears to contain gene expression data (they mention RNA was extracted from samples)

# Step 2: Variable Availability and Data Type Conversion

# 2.1 Data Availability
trait_row = 2  # Histology information can be used to determine lung cancer
age_row = 0    # Age at diagnosis is available
gender_row = 1 # Sex information is available

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert histology information to binary trait for lung cancer"""
    if not isinstance(value, str):
        return None
    
    # Extract value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Determine if the histology relates to lung cancer
    if 'adenocaricnoma' in value.lower():  # Lung adenocarcinoma
        return 1
    elif any(term in value.lower() for term in ['tnbc', 'er+', 'er-', 'pr+', 'pr-', 'her2', 'breast']):
        return 0  # Breast cancer types
    return None

def convert_age(value):
    """Convert age at diagnosis to numeric (continuous) value"""
    if not isinstance(value, str):
        return None
    
    # Extract value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Handle non-available or special cases
    if value.lower() in ['n.a.', 'n.a', 'unknown', 'na']:
        return None
    
    # Try to convert to numeric
    try:
        return float(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender information to binary (0 for female, 1 for male)"""
    if not isinstance(value, str):
        return None
    
    # Extract value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    return None

# Step 3: Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Step 4: Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical data:")
    print(preview)
    
    # Save the clinical data to the designated output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical data:
{'GSM7920782': [0.0, 49.0, 0.0], 'GSM7920783': [0.0, 44.0, 0.0], 'GSM7920784': [nan, 41.0, 0.0], 'GSM7920785': [0.0, 40.0, 0.0], 'GSM7920786': [0.0, 48.0, 0.0], 'GSM7920787': [nan, 42.0, 0.0], 'GSM7920788': [0.0, 47.0, 0.0], 'GSM7920789': [0.0, 53.0, 0.0], 'GSM7920790': [0.0, 41.0, 0.0], 'GSM7920791': [0.0, 74.0, 0.0], 'GSM7920792': [0.0, 58.0, 0.0], 'GSM7920793': [0.0, 51.0, 0.0], 'GSM7920794': [0.0, 55.0, 0.0], 'GSM7920795': [nan, 46.0, 0.0], 'GSM7920796': [0.0, 46.0, 0.0], 'GSM7920797': [0.0, 48.0, 0.0], 'GSM7920798': [0.0, 44.0, 0.0], 'GSM7920799': [0.0, 49.0, 0.0], 'GSM7920800': [0.0, 59.0, 0.0], 'GSM7920801': [0.0, 50.0, 0.0], 'GSM7920802': [0.0, 74.0, 0.0], 'GSM7920803': [0.0, 46.0, 0.0], 'GSM7920804': [1.0, 40.0, 0.0], 'GSM7920805': [1.0, 57.0, 1.0], 'GSM7920806': [1.0, 60.0, 1.0], 'GSM7920807': [1.0, 55.0, 0.0], 'GSM7920808': [1.0, 69.0, 0.0], 'GSM7920809': [1.0, nan, 0.0], 'GSM7920810': [1.0, nan, 1.0], 'GSM7920811': [1.0, 57.0, 1.0], 'GSM79

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Unlocking Molecular mechanisms and identifying druggable targets in matched-paired brain metastasis of Breast and Lung cancers "
Line 1: !Series_geo_accession	"GSE248830"
Line 2: !Series_status	"Public on Dec 03 2023"
Line 3: !Series_submission_date	"Nov 28 2023"
Line 4: !Series_last_update_date	"Dec 03 2023"
Line 5: !Series_summary	"Introduction: The incidence of brain metastases in cancer patients is increasing, with lung and breast cancer being the most common sources. Despite advancements in targeted therapies, the prognosis remains poor, highlighting the importance to investigate the underlying mechanisms in brain metastases. The aim of this study was to investigate the differences in the molecular mechanisms involved in brain metastasis of breast and lung cancers. In addition, we aimed to identify cancer lineage-specific druggable targets in the brain metastasis. Methods: To that aim, a cohort of 44 FFPE tissue samples, in

### Step 4: Gene Identifier Review

In [5]:
# Examining the first few gene identifiers from the data
# These look like standard human gene symbols (A2M, ACVR1C, ADAM12, etc.)
# These are not probe IDs, Ensembl IDs, or other identifiers that would require mapping

# Based on biomedical knowledge, these are standard HGNC gene symbols:
# A2M = Alpha-2-Macroglobulin
# ACVR1C = Activin A Receptor Type 1C
# ADAM12 = ADAM Metallopeptidase Domain 12
# etc.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols using NCBI Gene database
print("\nNormalizing gene symbols using NCBI Gene database...")
try:
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(gene_data_normalized)} unique genes remain")
    gene_data_cleaned = gene_data_normalized
    
    # Save the normalized gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_cleaned.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error during gene symbol normalization: {str(e)}")
    print("Falling back to original gene data")
    gene_data_cleaned = gene_data.copy()

# 2. We need to recreate the clinical data from the original matrix file
print("\nExtracting clinical data directly from the matrix file...")
# Extract clinical data from the matrix file again
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Process clinical data
print("\nProcessing clinical data...")
# Create clinical features dataframe 
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Extracted clinical data with shape: {selected_clinical_df.shape}")
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to: {out_clinical_data_file}")
    
    is_trait_available = True
else:
    selected_clinical_df = pd.DataFrame()
    is_trait_available = False
    print("No trait data available in clinical information.")

# 3. Link clinical and genetic data
if is_trait_available and is_gene_available:
    print("\nLinking clinical and genetic data...")
    try:
        # Ensure the sample IDs match between clinical and genetic data
        common_samples = list(set(selected_clinical_df.columns).intersection(set(gene_data_cleaned.columns)))
        
        if len(common_samples) == 0:
            print("Warning: No common samples between clinical and genetic data")
            linked_data = pd.DataFrame()
            is_biased = True
        else:
            print(f"Found {len(common_samples)} common samples between clinical and genetic data")
            
            # Filter data to include only common samples
            clinical_subset = selected_clinical_df[common_samples]
            genetic_subset = gene_data_cleaned[common_samples]
            
            # Link the data
            linked_data = pd.concat([clinical_subset, genetic_subset], axis=0).T
            print(f"Linked data shape: {linked_data.shape}")
            
            # 4. Handle missing values
            print("\nHandling missing values in linked data...")
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Data shape after handling missing values: {linked_data.shape}")
            
            # 5. Determine if trait and demographic features are severely biased
            print("\nEvaluating trait and demographic feature bias...")
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    except Exception as e:
        print(f"Error during data linking: {str(e)}")
        linked_data = pd.DataFrame()
        is_biased = True
else:
    print("\nCannot create linked data: missing clinical or gene data")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final validation and saving
note = "This dataset contains gene expression data from astrocytoma cell lines with modified GFAP isoform expression. The trait represents different experimental conditions related to the GFAPδ/GFAPα ratio."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if len(linked_data) > 0 else True,
    df=linked_data,
    note=note
)

# Save the linked data if it's usable
if is_usable and len(linked_data) > 0:
    print(f"\nSaving linked data to {out_data_file}")
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved successfully!")
else:
    print(f"\nDataset not usable for {trait} association studies due to bias or quality issues.")


Normalizing gene symbols using NCBI Gene database...
After normalization: 746 unique genes remain
Normalized gene expression data saved to ../../output/preprocess/Lung_Cancer/gene_data/GSE248830.csv

Extracting clinical data directly from the matrix file...

Processing clinical data...
Extracted clinical data with shape: (3, 44)
Clinical data saved to: ../../output/preprocess/Lung_Cancer/clinical_data/GSE248830.csv

Linking clinical and genetic data...
Found 44 common samples between clinical and genetic data
Linked data shape: (44, 749)

Handling missing values in linked data...
Data shape after handling missing values: (41, 749)

Evaluating trait and demographic feature bias...


For the feature 'Lung_Cancer', the least common label is '0.0' with 19 occurrences. This represents 46.34% of the dataset.
The distribution of the feature 'Lung_Cancer' in this dataset is fine.



Quartiles for 'Age':
  25%: 48.0
  50% (Median): 55.078947368421055
  75%: 60.0
Min: 37.0
Max: 74.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 12 occurrences. This represents 29.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.


Saving linked data to ../../output/preprocess/Lung_Cancer/GSE248830.csv
Linked data saved successfully!
